In [1]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.0 MB/s eta 0:00:00


In [13]:
import pandas as pd
import time
import json
import random
from Bio import Entrez, SeqIO
from collections import defaultdict
from tqdm.notebook import tqdm

# Set NCBI email (required)
Entrez.email = "bob@smith.com"  # Replace with your actual email

# Load the dataset from PathoPlexus
url = "https://lapis.pathoplexus.org/cchf/sample/details?downloadAsFile=true&downloadFileBasename=cchf_metadata_2025-04-03T2223&dataFormat=tsv&fields=accessionVersion%2CdataUseTerms%2CdataUseTermsUrl%2CearliestReleaseDate%2CgeoLocAdmin1%2CgeoLocAdmin2%2CgeoLocCity%2CgeoLocCountry%2ChostNameScientific%2Clength_L%2Clength_M%2Clength_S%2CsampleCollectionDate%2CinsdcAccessionFull_L%2CinsdcAccessionFull_S%2CinsdcAccessionFull_M%2CspecimenCollectorSampleId%2Cauthors%2CauthorAffiliations&versionStatus=LATEST_VERSION&isRevocation=false"
df = pd.read_csv(url, sep="\t")

# Gather all unique INSDC accessions
accession_cols = ["insdcAccessionFull_L", "insdcAccessionFull_M", "insdcAccessionFull_S"]
all_insdc_ids = set()
for col in accession_cols:
    all_insdc_ids.update(df[col].dropna().str.split(",").explode().str.strip())

# Remove any empty strings
all_insdc_ids = {acc for acc in all_insdc_ids if acc}

print(f"🔎 Found {len(all_insdc_ids)} unique INSDC accessions.")

# Function to fetch strain names from NCBI in batches using Biopython's SeqIO parser
def fetch_strains_batch(accessions, batch_size=100, max_retries=5, initial_delay=1, backoff_factor=2, jitter=0.1):
    results = {}
    # Process in batches
    for i in range(0, len(accessions), batch_size):
        batch = accessions[i:i+batch_size]
        retries = 0
        delay = initial_delay

        while retries <= max_retries:
            try:
                # Join accessions with comma for batch request
                acc_string = ",".join(batch)
                handle = Entrez.efetch(db="nucleotide", id=acc_string, rettype="gb", retmode="text")

                # Use Biopython's SeqIO to properly parse multiple records
                records = list(SeqIO.parse(handle, "genbank"))
                handle.close()

                print(f"Successfully retrieved {len(records)} records from batch of {len(batch)} accessions")

                # Extract strain information from each record
                for record in records:
                    accession = record.id

                    # Look for strain information in the features
                    strain = None
                    for feature in record.features:
                        if feature.type == "source" and "strain" in feature.qualifiers:
                            strain = feature.qualifiers["strain"][0]
                            break

                    if strain:
                        results[accession] = strain

                # If we got here, the batch was successful
                break

            except Exception as e:
                retries += 1
                if retries > max_retries:
                    print(f"Failed fetching batch after {max_retries} retries: {e}")
                    # Move on to the next batch
                    break

                # Calculate backoff with jitter
                jitter_amount = random.uniform(-jitter * delay, jitter * delay)
                sleep_time = delay + jitter_amount

                print(f"Error fetching batch: {e}. Retry {retries}/{max_retries} after {sleep_time:.2f}s")
                time.sleep(sleep_time)

                # Increase delay for next retry using exponential backoff
                delay *= backoff_factor

        # Add polite delay between batches to be nice to NCBI
        time.sleep(0.5)

    return results

print("⚡ Fetching strain names using NCBI efetch with batch processing...")
accession_list = list(all_insdc_ids)
batch_size = 200  # Using a batch size of 100 (within NCBI's guidelines)

with tqdm(total=len(all_insdc_ids), desc="Batch fetching") as pbar:
    accession_to_strain = {}

    # Process in batches of batch_size
    for i in range(0, len(accession_list), batch_size):
        batch = accession_list[i:i+batch_size]
        batch_results = fetch_strains_batch(batch, batch_size=batch_size)
        accession_to_strain.update(batch_results)
        pbar.update(len(batch))

    print(f"Batch processing completed. Found strain info for {len(accession_to_strain)} accessions.")

# Map strain → list of accessions
strain_to_accessions = defaultdict(list)
for acc, strain in accession_to_strain.items():
    if strain:  # Only add items with actual strain information
        strain_to_accessions[strain].append(acc)

# Save result to JSON
with open("strain_accession_map.json", "w") as f:
    json.dump(strain_to_accessions, f, indent=2)

# Summary statistics
print(f"✅ Processed {len(all_insdc_ids)} accessions")
print(f"✅ Found strain information for {len(accession_to_strain)} accessions")
print(f"✅ Identified {len(strain_to_accessions)} unique strains")
print("✅ All done! Saved as 'strain_accession_map.json'")

🔎 Found 4800 unique INSDC accessions.
⚡ Fetching strain names using NCBI efetch with batch processing...


Batch fetching:   0%|          | 0/4800 [00:00<?, ?it/s]

Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from batch of 200 accessions
Successfully retrieved 200 records from 

Successfully retrieved 1 records from batch of 1 accessions


{'PP894733.1': 'SH401971'}

NameError: name 'records' is not defined

defaultdict(list, {})

('PP894733.1', 'SH401971')

In [14]:
import pandas as pd
import json
import time
from collections import defaultdict

# ----- Step 1: Load strain→accession map -----
with open("strain_accession_map.json", "r") as f:
    strain_accession_map = json.load(f)

# Create reverse lookup
accession_to_strain = {}
for strain, accessions in strain_accession_map.items():
    for acc in accessions:
        accession_to_strain[acc] = strain

# ----- Step 2: Download input dataset -----
url = (
    "https://lapis.pathoplexus.org/cchf/sample/details?"
    "downloadAsFile=true&downloadFileBasename=cchf_metadata_2025-04-03T2223"
    "&dataFormat=tsv&fields=accessionVersion%2CdataUseTerms%2CdataUseTermsUrl"
    "%2CearliestReleaseDate%2CgeoLocAdmin1%2CgeoLocAdmin2%2CgeoLocCity"
    "%2CgeoLocCountry%2ChostNameScientific%2Clength_L%2Clength_M%2Clength_S"
    "%2CsampleCollectionDate%2CinsdcAccessionFull_L%2CinsdcAccessionFull_S"
    "%2CinsdcAccessionFull_M%2CspecimenCollectorSampleId%2Cauthors%2CauthorAffiliations"
    "&versionStatus=LATEST_VERSION&isRevocation=false"
)
df = pd.read_csv(url, sep="\t")

# ----- Step 3: Segment presence flags -----
df["has_L"] = df["length_L"].apply(lambda x: x > 0)
df["has_M"] = df["length_M"].apply(lambda x: x > 0)
df["has_S"] = df["length_S"].apply(lambda x: x > 0)

# ----- Step 4: Assign strain per row -----
def determine_strain(row):
    for col in ["insdcAccessionFull_L", "insdcAccessionFull_M", "insdcAccessionFull_S"]:
        if pd.notnull(row[col]):
            ids = [x.strip() for x in row[col].split(",") if x.strip()]
            for acc in ids:
                if acc in accession_to_strain:
                    return accession_to_strain[acc]
    return None

df["strain"] = df.apply(determine_strain, axis=1)
df = df[df["strain"].notnull()].copy()
print(f"✅ Rows with matched strain info: {len(df)}")

# ----- Step 5: Group by strain -----
grouped_by_strain = df.groupby("strain").agg({
    "length_L": "max",
    "length_M": "max",
    "length_S": "max",
    "accessionVersion": lambda x: list(x),
    "insdcAccessionFull_L": lambda x: ", ".join(x.dropna().astype(str)),
    "insdcAccessionFull_M": lambda x: ", ".join(x.dropna().astype(str)),
    "insdcAccessionFull_S": lambda x: ", ".join(x.dropna().astype(str)),
    "sampleCollectionDate": lambda x: ", ".join(x.dropna().astype(str)),
    "geoLocCountry": lambda x: ", ".join(x.dropna().astype(str)),
    "hostNameScientific": lambda x: ", ".join(x.dropna().astype(str)),
    "authors": lambda x: list(x),
    "earliestReleaseDate": lambda x: ", ".join(x.dropna().astype(str)),
    "has_L": "sum",
    "has_M": "sum",
    "has_S": "sum"
}).reset_index()

# ----- Step 6: Filter groups -----
filtered = grouped_by_strain[
    (grouped_by_strain["accessionVersion"].apply(lambda x: len(x) > 1)) &
    (grouped_by_strain["has_L"] <= 1) &
    (grouped_by_strain["has_M"] <= 1) &
    (grouped_by_strain["has_S"] <= 1)
]

# ----- Step 7: Generate JSON output -----
def extract_insdc_list(row):
    insdc_ids = []
    for col in ["insdcAccessionFull_L", "insdcAccessionFull_M", "insdcAccessionFull_S"]:
        if pd.notnull(row[col]):
            insdc_ids.extend([x.strip() for x in row[col].split(",") if x.strip()])
    return insdc_ids

strain_json = {}
for _, row in filtered.iterrows():
    insdc_ids = extract_insdc_list(row)
    unique_ids = list(dict.fromkeys(insdc_ids))  # keep order, remove dups
    key = "_".join(unique_ids)
    strain_json[key] = insdc_ids

# ----- Step 8: Save outputs -----
filtered.to_csv("grouped_by_strain.tsv", sep="\t", index=False)
with open("strain_groupings.json", "w") as f:
    json.dump(strain_json, f, indent=2)

print("✅ Saved:")
print("- grouped_by_strain.tsv")
print("- strain_groupings.json")


✅ Rows with matched strain info: 589
✅ Saved:
- grouped_by_strain.tsv
- strain_groupings.json
